In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
import warnings
warnings.filterwarnings('ignore')

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 3
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.width'] = 2

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/gabor/'
path_data = main_path+'/data/jatos_gabor_data/tanda1/'

In [ ]:
fday = 10
fsession = 2

path = path_data+'day'+str(fday)+'/session'+str(fsession)+'/'
filename_average=path_results+'across_sessions/average_Day'+str(fday)+'_Sess'+str(fsession)+'.json'

In [ ]:
data_files = [f for f in os.listdir(path) if f.endswith('_day'+str(fday)+'_session'+str(fsession))]

In [ ]:
# sort files
subj_data = [int(re.search('%s(.*)%s' % ('', '_day'), f).group(1)) for f in data_files]
sorted_subj_data = sorted(subj_data)
index_subj_data = [subj_data.index(elem) for elem in sorted_subj_data]
sorted_data_files = [data_files[i] for i in index_subj_data]

In [ ]:
# reports, staircase, deterministic and stochastic df for each participant

Rdf,Cdf,Ddf,Sdf = {},{},{},{}
for name in sorted_subj_data: 
    Rdf[name] = {}
    Cdf[name] = pd.DataFrame()
    Ddf[name] = pd.DataFrame()
    Sdf[name] = pd.DataFrame()

In [ ]:
ind = -1
for ses in sorted_data_files:
    ind += 1
    data = [] 
    for line in open(path+ses, 'r'):
        if line.strip():
            data.append(json.loads(line))

    if fday==1 and fsession==1:
        if len(data)==6:
            demographics = data[0]
            reports = data[1]
            practice = data[2]
            staircase = data[3]
            deterministic = data[4]
            stochastic = data[5]
        else:
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('this participant has repeated some stage')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
    else:
        if len(data)==4:
            reports = data[0]
            staircase = data[1]
            deterministic = data[2]
            stochastic = data[3]
        elif len(data)>4:
            reports = data[0]
            staircase = data[len(data)-3]
            deterministic = data[len(data)-2]
            stochastic = data[len(data)-1]
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('participant '+ str(sorted_subj_data[ind])+' has repeated staircase '+str(len(data)-3))
            stair = []
            for j in range(len(data)):
                if len(data[j])==60:
                    stair.append(data[j])
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
        else: 
            for dd in range(len(data)):
                print(len(data[dd]))
            reports = data[0]
            staircase = data[1]
            deterministic = data[2]
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('participant '+ str(sorted_subj_data[ind])+' missed stochastic')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')
            print('!!!!!!!!!!!!!!!!!!')           
            
        
    part = reports['userID']
    if int(part)>1015 and int(part)<1200:
        part = str(int(part)-186)
    elif int(part)>1200:
        part = str(int(part)-217)
    if reports['sessionID']%2==0:
        session = 2
    else:
        session = 1
    
    if (part!=str(sorted_subj_data[ind])):
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('INCORRECT')
        print('participant',part,'file participant',sorted_subj_data[ind])
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
    if (session!=fsession):
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('INCORRECT')
        print('session',session,'file session',fsession)
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')

    Rdf[sorted_subj_data[ind]] = reports
    Cdf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(staircase)
    Ddf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(deterministic)
    Sdf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(stochastic)
    print('participant: ',part,'initial datetime stamp: ',reports['date'])

In [ ]:
# path to save results
path_win = path_results+'winners.csv'

In [ ]:
total_score = []
for part in sorted_subj_data:
    total_score.append(np.sum(Ddf[part]['score'])+np.sum(Sdf[part]['score']))
    print(part,np.sum(Ddf[part]['score'])+np.sum(Sdf[part]['score']))

In [ ]:
winners = np.zeros(4)
winners[0] = fday
winners[1] = fsession

In [ ]:
sorted_total_score = sorted(total_score, reverse=True)
print(sorted_total_score)
print(sorted_subj_data[total_score.index(sorted_total_score[0])],\
      sorted_subj_data[total_score.index(sorted_total_score[1])])
print(np.where(total_score==sorted_total_score[0]))
if len(np.where(total_score==sorted_total_score[0])[0])==2:
    winners[2] = sorted_subj_data[np.where(total_score==sorted_total_score[0])[0][0]]
    winners[3] = sorted_subj_data[np.where(total_score==sorted_total_score[0])[0][1]]
elif len(np.where(total_score==sorted_total_score[0])[0])>2:
    print('más de dos ganadores!!!')
else:
    winners[2] = sorted_subj_data[total_score.index(sorted_total_score[0])]
    winners[3] = sorted_subj_data[total_score.index(sorted_total_score[1])]

In [ ]:
from csv import writer 
with open(path_win, 'a') as f: 
    # Pass this file object to csv.writer() 
    # and get a writer object 
    writer_object = writer(f) 
    # Pass the list as an argument into 
    # the writerow() 
    writer_object.writerow(winners) 
    #Close the file object 
    f.close() 

In [ ]:
Wdf = pd.read_csv(path_win) 

In [ ]:
win1 = list(Wdf['win1'])
win2 = list(Wdf['win2'])
win = win1+win2

In [ ]:
win_freq = {x:win.count(x) for x in win}
print(win_freq)

In [ ]:
for k,v in win_freq.items():
    print(int(k),v)
    print('\n')